# Supermind Data Science Assignment  : Crypto Detection 

### Name : Devang Papinwar
### Email : papinwardevang@gmail.com
### Contact No : 7420039018

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import string
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import contractions
from math import sqrt, pow, exp
from sklearn.feature_extraction.text import CountVectorizer
sns.set()
import spacy
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()# Visualise inside a notebook
import en_core_web_md
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel

/opt/anaconda3/lib/python3.9/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


### Preparing Dataset

In [2]:
definition = pd.read_csv('term_def.csv')
definition

,terms,definition1,definition2
0,51% attack,A hypothetical situation where more than half ...,NaN
1,51% attack protection,A protection mechanism implemented by several ...,NaN
2,AFK,Away From Keyboard; used on social media platf...,NaN
3,Airdrop,An event where a blockchain project distribute...,NaN
4,Altcoin,Any cryptocurrency that is an alternative to B...,NaN
...,...,...,...
155,Vyper,A Python-like programming language for the Eth...,NaN
156,Wallet (Cold),A wallet disconnected from the internet.,NaN
157,Wallet (Hot),A wallet connected to the Internet.,NaN
158,Wallet (Multisignature),A wallet that requires multiple digital signat...,NaN


In [3]:
data = pd.DataFrame()
data['sentence'] = definition['definition1']
data

,sentence
0,A hypothetical situation where more than half ...
1,A protection mechanism implemented by several ...
2,Away From Keyboard; used on social media platf...
3,An event where a blockchain project distribute...
4,Any cryptocurrency that is an alternative to B...
...,...
155,A Python-like programming language for the Eth...
156,A wallet disconnected from the internet.
157,A wallet connected to the Internet.
158,A wallet that requires multiple digital signat...


### Preprocess the data

In [4]:
def text_preprocessing(text):
    # Removing contents of tags and all for further text processing
    data['sentence'].replace(regex=True,inplace=True, to_replace= r'<.+?>', value=r' ')
    
    # Removing the \t charecter from the descriptions field
    data['sentence'] = data['sentence'].str.replace(r'\n', ' ')
    
    data['sentence'] = data['sentence'].str.replace(r'  ', '')

    # Removing links from all for further text processing
    data['sentence'].replace(regex=True,inplace=True, to_replace= r'(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*', value=r' ')
    
    # Remove punctuation
    data['sentence'].replace(regex=True,inplace=True, to_replace= r'[^\w\d\s]', value=r'')
    
    # Removing rows with empty columns 
    data.dropna(subset=['sentence'],inplace=True)
    
    return data

In [5]:
data = text_preprocessing(data)
data

/var/folders/23/12jln18952d9zg137wpr4bq80000gn/T/ipykernel_28455/4089952422.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  data['sentence'] = data['sentence'].str.replace(r'\n', ' ')


,sentence
0,A hypothetical situation where more than half ...
1,A protection mechanism implemented by several ...
2,Away From Keyboard used on social media platfo...
3,An event where a blockchain project distribute...
4,Any cryptocurrency that is an alternative to B...
...,...
155,A Pythonlike programming language for the Ethe...
156,A wallet disconnected from the internet
157,A wallet connected to the Internet
158,A wallet that requires multiple digital signat...


### Preprocess the Data removing stopwords , tokenization , lowering charecters

In [6]:
# spacy model:
nlp = en_core_web_md.load()

# Tags to remove from the text
removal= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']
tokens = []

for summary in nlp.pipe(data['sentence']):
   proj_tok = [token.lemma_.lower() for token in summary if token.pos_ not in removal and not token.is_stop and token.is_alpha]
   tokens.append(proj_tok)
    
tokens

[['hypothetical',
  'situation',
  'half',
  'computing',
  'power',
  'blockchain',
  'network',
  'control',
  'person',
  'group',
  'allow',
  'dictate',
  'transaction',
  'verify',
  'allow',
  'prevent',
  'user',
  'complete',
  'confirm',
  'transaction',
  'cause',
  'havoc',
  'system',
  'doublespend',
  'coin'],
 ['protection',
  'mechanism',
  'implement',
  'cryptocurrencie',
  'require',
  'total',
  'hashing',
  'power',
  'work',
  'entity',
  'difficult',
  'attacker',
  'd',
  'need',
  'resource',
  'time',
  'threshold',
  'have',
  'additional',
  'safeguard',
  'feature',
  'agree',
  'transaction',
  'send',
  'make',
  'unable',
  'doublespend',
  'notice',
  'change',
  'chain'],
 ['keyboard',
  'social',
  'medium',
  'platform',
  'like',
  'twitter',
  'user',
  'share',
  'trading',
  'activity',
  'want',
  'receive',
  'message',
  'log',
  'account',
  'thing',
  'afk',
  'trade',
  'extended',
  'period',
  'time',
  'active',
  'feed'],
 ['event',
  

### Create a new column for tokens

In [7]:
data["tokens"] = tokens
data

,sentence,tokens
0,A hypothetical situation where more than half ...,"[hypothetical, situation, half, computing, pow..."
1,A protection mechanism implemented by several ...,"[protection, mechanism, implement, cryptocurre..."
2,Away From Keyboard used on social media platfo...,"[keyboard, social, medium, platform, like, twi..."
3,An event where a blockchain project distribute...,"[event, blockchain, project, distribute, free,..."
4,Any cryptocurrency that is an alternative to B...,"[cryptocurrency, alternative, bitcoin]"
...,...,...
155,A Pythonlike programming language for the Ethe...,"[pythonlike, programming, language, ethereum, ..."
156,A wallet disconnected from the internet,"[wallet, disconnect, internet]"
157,A wallet connected to the Internet,"[wallet, connect, internet]"
158,A wallet that requires multiple digital signat...,"[wallet, require, multiple, digital, signature..."


### Create a Dictionary from the Bag of Words

In [8]:
dictionary = Dictionary(data['tokens'])
print(dictionary.token2id)

{'allow': 0, 'blockchain': 1, 'cause': 2, 'coin': 3, 'complete': 4, 'computing': 5, 'confirm': 6, 'control': 7, 'dictate': 8, 'doublespend': 9, 'group': 10, 'half': 11, 'havoc': 12, 'hypothetical': 13, 'network': 14, 'person': 15, 'power': 16, 'prevent': 17, 'situation': 18, 'system': 19, 'transaction': 20, 'user': 21, 'verify': 22, 'additional': 23, 'agree': 24, 'attacker': 25, 'chain': 26, 'change': 27, 'cryptocurrencie': 28, 'd': 29, 'difficult': 30, 'entity': 31, 'feature': 32, 'hashing': 33, 'have': 34, 'implement': 35, 'make': 36, 'mechanism': 37, 'need': 38, 'notice': 39, 'protection': 40, 'require': 41, 'resource': 42, 'safeguard': 43, 'send': 44, 'threshold': 45, 'time': 46, 'total': 47, 'unable': 48, 'work': 49, 'account': 50, 'active': 51, 'activity': 52, 'afk': 53, 'extended': 54, 'feed': 55, 'keyboard': 56, 'like': 57, 'log': 58, 'medium': 59, 'message': 60, 'period': 61, 'platform': 62, 'receive': 63, 'share': 64, 'social': 65, 'thing': 66, 'trade': 67, 'trading': 68, 'tw

In [9]:
# Generate a corpus from the BagofWords Dictionary
corpus = [dictionary.doc2bow(doc) for doc in data['tokens']]
corpus

[[(0, 2),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 2),
  (21, 1),
  (22, 1)],
 [(9, 1),
  (16, 1),
  (20, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1)],
 [(21, 1),
  (46, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1)],
 [(1, 1), (3, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1)],
 [(77, 1), (78, 1), (79, 1)],
 [(57, 1),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 1),
  (84, 1),
  (

### Create a Model for topic modeling ie. Latent Dirichlet Allocation

In [10]:
# Train the unsupervised machine learning model on the data
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=50, num_topics=10, workers = 4, passes=10)

In [11]:
# Print the topics modeled from the data
lda_model.print_topics(0)

[(0,
  '0.040*"blockchain" + 0.014*"investor" + 0.014*"financial" + 0.014*"price" + 0.009*"application" + 0.009*"business" + 0.009*"process" + 0.009*"customer" + 0.009*"launder" + 0.009*"production"'),
 (7,
  '0.030*"coin" + 0.022*"blockchain" + 0.020*"token" + 0.017*"digital" + 0.014*"wallet" + 0.014*"term" + 0.014*"slang" + 0.014*"market" + 0.014*"cryptocurrency" + 0.011*"hold"'),
 (5,
  '0.018*"blockchain" + 0.012*"transaction" + 0.012*"network" + 0.012*"order" + 0.012*"sell" + 0.009*"improve" + 0.009*"security" + 0.009*"short" + 0.009*"attack" + 0.009*"increase"'),
 (8,
  '0.023*"system" + 0.019*"transaction" + 0.012*"buy" + 0.012*"product" + 0.012*"create" + 0.012*"like" + 0.011*"investor" + 0.008*"occur" + 0.008*"node" + 0.008*"network"'),
 (4,
  '0.020*"transaction" + 0.015*"value" + 0.012*"blockchain" + 0.012*"example" + 0.012*"item" + 0.009*"asset" + 0.009*"cryptocurrencie" + 0.009*"place" + 0.009*"market" + 0.009*"condition"'),
 (1,
  '0.025*"transaction" + 0.025*"blockchain"

In [12]:
# Visualising the topics modeled from the data
lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(lda_display)

/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: t

### Create a Bag of Words from the Latent Dirichlet Allocation's modeled topics

In [13]:
# For each topic modeled show the high probability words
for idx, topic in lda_model.print_topics(0):
   print('Topic: {} \nWords: {}'.format(idx, topic))

# Generate the number of sub topics for main topics generated
topics = lda_model.print_topics(idx, 20)
topics

Topic: 4 
Words: 0.020*"transaction" + 0.015*"value" + 0.012*"blockchain" + 0.012*"example" + 0.012*"item" + 0.009*"asset" + 0.009*"cryptocurrencie" + 0.009*"place" + 0.009*"market" + 0.009*"condition"
Topic: 8 
Words: 0.023*"system" + 0.019*"transaction" + 0.012*"buy" + 0.012*"product" + 0.012*"create" + 0.012*"like" + 0.011*"investor" + 0.008*"occur" + 0.008*"node" + 0.008*"network"
Topic: 5 
Words: 0.018*"blockchain" + 0.012*"transaction" + 0.012*"network" + 0.012*"order" + 0.012*"sell" + 0.009*"improve" + 0.009*"security" + 0.009*"short" + 0.009*"attack" + 0.009*"increase"
Topic: 7 
Words: 0.030*"coin" + 0.022*"blockchain" + 0.020*"token" + 0.017*"digital" + 0.014*"wallet" + 0.014*"term" + 0.014*"slang" + 0.014*"market" + 0.014*"cryptocurrency" + 0.011*"hold"
Topic: 1 
Words: 0.025*"transaction" + 0.025*"blockchain" + 0.014*"system" + 0.014*"control" + 0.014*"network" + 0.011*"process" + 0.011*"central" + 0.009*"exchange" + 0.009*"use" + 0.009*"security"
Topic: 9 
Words: 0.029*"sma

[(9,
  '0.029*"smart" + 0.029*"contract" + 0.023*"token" + 0.022*"function" + 0.019*"key" + 0.016*"ethereum" + 0.015*"private" + 0.013*"transaction" + 0.012*"total" + 0.012*"wallet" + 0.012*"support" + 0.009*"include" + 0.009*"give" + 0.008*"datum" + 0.008*"unique" + 0.008*"message" + 0.008*"nonfungible" + 0.008*"high" + 0.008*"asset" + 0.008*"research"'),
 (2,
  '0.029*"block" + 0.029*"miner" + 0.017*"blockchain" + 0.015*"mining" + 0.013*"currency" + 0.013*"network" + 0.011*"process" + 0.011*"burn" + 0.011*"identity" + 0.011*"node" + 0.011*"verify" + 0.010*"cryptocurrency" + 0.009*"stake" + 0.009*"pow" + 0.009*"work" + 0.008*"transaction" + 0.007*"exchange" + 0.007*"form" + 0.007*"allow" + 0.007*"application"'),
 (4,
  '0.020*"transaction" + 0.015*"value" + 0.012*"blockchain" + 0.012*"example" + 0.012*"item" + 0.009*"asset" + 0.009*"cryptocurrencie" + 0.009*"place" + 0.009*"market" + 0.009*"condition" + 0.009*"group" + 0.009*"token" + 0.009*"record" + 0.009*"property" + 0.009*"like" +

### Clean the Bag of Words

In [14]:
topics1 = []
for idx , topic in topics:
    topics1.append(topic.split('+'))
topics1

[['0.029*"smart" ',
  ' 0.029*"contract" ',
  ' 0.023*"token" ',
  ' 0.022*"function" ',
  ' 0.019*"key" ',
  ' 0.016*"ethereum" ',
  ' 0.015*"private" ',
  ' 0.013*"transaction" ',
  ' 0.012*"total" ',
  ' 0.012*"wallet" ',
  ' 0.012*"support" ',
  ' 0.009*"include" ',
  ' 0.009*"give" ',
  ' 0.008*"datum" ',
  ' 0.008*"unique" ',
  ' 0.008*"message" ',
  ' 0.008*"nonfungible" ',
  ' 0.008*"high" ',
  ' 0.008*"asset" ',
  ' 0.008*"research"'],
 ['0.029*"block" ',
  ' 0.029*"miner" ',
  ' 0.017*"blockchain" ',
  ' 0.015*"mining" ',
  ' 0.013*"currency" ',
  ' 0.013*"network" ',
  ' 0.011*"process" ',
  ' 0.011*"burn" ',
  ' 0.011*"identity" ',
  ' 0.011*"node" ',
  ' 0.011*"verify" ',
  ' 0.010*"cryptocurrency" ',
  ' 0.009*"stake" ',
  ' 0.009*"pow" ',
  ' 0.009*"work" ',
  ' 0.008*"transaction" ',
  ' 0.007*"exchange" ',
  ' 0.007*"form" ',
  ' 0.007*"allow" ',
  ' 0.007*"application"'],
 ['0.020*"transaction" ',
  ' 0.015*"value" ',
  ' 0.012*"blockchain" ',
  ' 0.012*"example" ',
 

In [15]:
topics2 = []
for tapic in topics1:
    for tap in tapic:
        topics2.append(tap.split('*'))
topics2

[['0.029', '"smart" '],
 [' 0.029', '"contract" '],
 [' 0.023', '"token" '],
 [' 0.022', '"function" '],
 [' 0.019', '"key" '],
 [' 0.016', '"ethereum" '],
 [' 0.015', '"private" '],
 [' 0.013', '"transaction" '],
 [' 0.012', '"total" '],
 [' 0.012', '"wallet" '],
 [' 0.012', '"support" '],
 [' 0.009', '"include" '],
 [' 0.009', '"give" '],
 [' 0.008', '"datum" '],
 [' 0.008', '"unique" '],
 [' 0.008', '"message" '],
 [' 0.008', '"nonfungible" '],
 [' 0.008', '"high" '],
 [' 0.008', '"asset" '],
 [' 0.008', '"research"'],
 ['0.029', '"block" '],
 [' 0.029', '"miner" '],
 [' 0.017', '"blockchain" '],
 [' 0.015', '"mining" '],
 [' 0.013', '"currency" '],
 [' 0.013', '"network" '],
 [' 0.011', '"process" '],
 [' 0.011', '"burn" '],
 [' 0.011', '"identity" '],
 [' 0.011', '"node" '],
 [' 0.011', '"verify" '],
 [' 0.010', '"cryptocurrency" '],
 [' 0.009', '"stake" '],
 [' 0.009', '"pow" '],
 [' 0.009', '"work" '],
 [' 0.008', '"transaction" '],
 [' 0.007', '"exchange" '],
 [' 0.007', '"form

In [16]:
tops = []

for tip in topics2:   
    tops.append(tip[1][1:-2])
tops

['smart',
 'contract',
 'token',
 'function',
 'key',
 'ethereum',
 'private',
 'transaction',
 'total',
 'wallet',
 'support',
 'include',
 'give',
 'datum',
 'unique',
 'message',
 'nonfungible',
 'high',
 'asset',
 'researc',
 'block',
 'miner',
 'blockchain',
 'mining',
 'currency',
 'network',
 'process',
 'burn',
 'identity',
 'node',
 'verify',
 'cryptocurrency',
 'stake',
 'pow',
 'work',
 'transaction',
 'exchange',
 'form',
 'allow',
 'applicatio',
 'transaction',
 'value',
 'blockchain',
 'example',
 'item',
 'asset',
 'cryptocurrencie',
 'place',
 'market',
 'condition',
 'group',
 'token',
 'record',
 'property',
 'like',
 'usd',
 'digital',
 'signature',
 'multiple',
 'bitcoi',
 'system',
 'transaction',
 'buy',
 'product',
 'create',
 'like',
 'investor',
 'occur',
 'node',
 'network',
 'sell',
 'decision',
 'connect',
 'protocol',
 'share',
 'party',
 'purpose',
 'verify',
 'file',
 'devic',
 'blockchain',
 'investor',
 'financial',
 'price',
 'application',
 'business'

### Getting the terms from the abbrevations part

In [17]:
terms = []

for term in definition['terms']:
    terms.append(term.split(' '))
terms

[['51%', 'attack'],
 ['51%', 'attack', 'protection'],
 ['AFK'],
 ['Airdrop'],
 ['Altcoin'],
 ['AML'],
 ['ATH'],
 ['Bearish'],
 ['Bear', 'market'],
 ['Bitcoin'],
 ['Blockchain'],
 ['Block', 'height'],
 ['Block', 'reward'],
 ['Bollinger', 'bands'],
 ['BTD'],
 ['Bullish'],
 ['Casper'],
 ['Centralised'],
 ['Coinless', 'protocol'],
 ['Chaffing'],
 ['Confirmation'],
 ['Crowdsale'],
 ['Cryptoart'],
 ['Cryptocurrency'],
 ['Crypto', 'derivatives'],
 ['Cryptoeconomics'],
 ['Cryptography'],
 ['Decentralised'],
 ['Distribution'],
 ['Distributed', 'ledger'],
 ['Digital', 'gold'],
 ['Dumping'],
 ['DYOR'],
 ['Entry', 'and', 'exit', 'points'],
 ['Etherscan'],
 ['Fear', 'and', 'greed', 'index'],
 ['Fiat', 'gateways'],
 ['Flippening'],
 ['FOMO'],
 ['Fork'],
 ['FUD'],
 ['FUDster'],
 ['Futures'],
 ['Gas'],
 ['Halving'],
 ['Hash', 'function'],
 ['Hedging'],
 ['HODL'],
 ['Hot', 'wallet'],
 ['IDO'],
 ['IEO'],
 ['Inflation'],
 ['KYC'],
 ['Lambo'],
 ['Memecoin'],
 ['Mimblewimble'],
 ['Miner'],
 ['Mining'],
 ['

In [18]:
terms1 = []
for term in terms:
    for t in term:
        terms1.append(t.lower())
        
terms1

['51%',
 'attack',
 '51%',
 'attack',
 'protection',
 'afk',
 'airdrop',
 'altcoin',
 'aml',
 'ath',
 'bearish',
 'bear',
 'market',
 'bitcoin',
 'blockchain',
 'block',
 'height',
 'block',
 'reward',
 'bollinger',
 'bands',
 'btd',
 'bullish',
 'casper',
 'centralised',
 'coinless',
 'protocol',
 'chaffing',
 'confirmation',
 'crowdsale',
 'cryptoart',
 'cryptocurrency',
 'crypto',
 'derivatives',
 'cryptoeconomics',
 'cryptography',
 'decentralised',
 'distribution',
 'distributed',
 'ledger',
 'digital',
 'gold',
 'dumping',
 'dyor',
 'entry',
 'and',
 'exit',
 'points',
 'etherscan',
 'fear',
 'and',
 'greed',
 'index',
 'fiat',
 'gateways',
 'flippening',
 'fomo',
 'fork',
 'fud',
 'fudster',
 'futures',
 'gas',
 'halving',
 'hash',
 'function',
 'hedging',
 'hodl',
 'hot',
 'wallet',
 'ido',
 'ieo',
 'inflation',
 'kyc',
 'lambo',
 'memecoin',
 'mimblewimble',
 'miner',
 'mining',
 'mining',
 'difficulty',
 'mining',
 'rigs',
 'moon',
 'nft',
 'node',
 'peer',
 'to',
 'peer',
 '

### Adding them to the Final BagOfWords

In [19]:
data = tops.extend(terms1)

for term in terms1:
    tops.append(term)

tops

['smart',
 'contract',
 'token',
 'function',
 'key',
 'ethereum',
 'private',
 'transaction',
 'total',
 'wallet',
 'support',
 'include',
 'give',
 'datum',
 'unique',
 'message',
 'nonfungible',
 'high',
 'asset',
 'researc',
 'block',
 'miner',
 'blockchain',
 'mining',
 'currency',
 'network',
 'process',
 'burn',
 'identity',
 'node',
 'verify',
 'cryptocurrency',
 'stake',
 'pow',
 'work',
 'transaction',
 'exchange',
 'form',
 'allow',
 'applicatio',
 'transaction',
 'value',
 'blockchain',
 'example',
 'item',
 'asset',
 'cryptocurrencie',
 'place',
 'market',
 'condition',
 'group',
 'token',
 'record',
 'property',
 'like',
 'usd',
 'digital',
 'signature',
 'multiple',
 'bitcoi',
 'system',
 'transaction',
 'buy',
 'product',
 'create',
 'like',
 'investor',
 'occur',
 'node',
 'network',
 'sell',
 'decision',
 'connect',
 'protocol',
 'share',
 'party',
 'purpose',
 'verify',
 'file',
 'devic',
 'blockchain',
 'investor',
 'financial',
 'price',
 'application',
 'business'

### Final Bag of Words

In [20]:
tops

['smart',
 'contract',
 'token',
 'function',
 'key',
 'ethereum',
 'private',
 'transaction',
 'total',
 'wallet',
 'support',
 'include',
 'give',
 'datum',
 'unique',
 'message',
 'nonfungible',
 'high',
 'asset',
 'researc',
 'block',
 'miner',
 'blockchain',
 'mining',
 'currency',
 'network',
 'process',
 'burn',
 'identity',
 'node',
 'verify',
 'cryptocurrency',
 'stake',
 'pow',
 'work',
 'transaction',
 'exchange',
 'form',
 'allow',
 'applicatio',
 'transaction',
 'value',
 'blockchain',
 'example',
 'item',
 'asset',
 'cryptocurrencie',
 'place',
 'market',
 'condition',
 'group',
 'token',
 'record',
 'property',
 'like',
 'usd',
 'digital',
 'signature',
 'multiple',
 'bitcoi',
 'system',
 'transaction',
 'buy',
 'product',
 'create',
 'like',
 'investor',
 'occur',
 'node',
 'network',
 'sell',
 'decision',
 'connect',
 'protocol',
 'share',
 'party',
 'purpose',
 'verify',
 'file',
 'devic',
 'blockchain',
 'investor',
 'financial',
 'price',
 'application',
 'business'

### User Defined function for generating tokens

In [21]:
def tokenization(text):
    tokens = (word_tokenize(text))
    return tokens

### User Defined function for removing stopwords

In [22]:
def remove_stopwords(text):
    words = [w for w in word_tokens if not w.lower() in stop_words]
    return words

### User Defined function for getting cosine similarity score

In [23]:
def cosine(rvector , c):
    for i in range(len(rvector)):
        c+= l1[i]*l2[i]
    if int(c) == 0:
        return 0
    return c / float((sum(l1)*sum(l2))**0.5)

### Generate a Test dataset from multiple datasets

In [51]:
# Generate various datasets to test against
art = pd.read_csv('art_results.csv')
music = pd.read_csv('music_results.csv')
sport = pd.read_csv('sport_results.csv')
news = pd.read_csv('news_results.csv')
food = pd.read_csv('food_results.csv')
crypto = pd.read_csv('crypto_results.csv')

In [52]:
# We know the art , music , sport and news contains sentences not related to blockchain , crypto and web3
data1 = pd.DataFrame()
data1['sentence'] = art['sentence'][:300]
data1

data2 = pd.DataFrame()
data2['sentence'] = music['sentence'][:300]
data2

data3 = pd.DataFrame()
data3['sentence'] = sport['sentence'][:300]
data3

data4 = pd.DataFrame()
data4['sentence'] = news['sentence'][:300]
data4

data5 = pd.DataFrame()
data5['sentence'] = food['sentence'][:300]
data5

# We know the crypto dataset contains sentences related to blockchain , crypto and web3
data6 = pd.DataFrame()
data6['sentence'] = crypto['sentence'][:300]
data6

,sentence
0,welcome to this site was founded in may 2013 ...
1,here at we work very hard to ensure that all ...
2,each of our coin data pages has a graph that s...
3,we receive updated prices directly from many ...
4,related links new to crypto learn how to buy b...
...,...
295,the term electronic commerce refers to any fin...
296,we will refer to the storage medium as a card ...
297,a particular kind of electronic commerce is th...
298,with the rise of and the internet it is the ...


In [53]:
# Create a Final Test Data
testData = pd.DataFrame()

testData = testData.append(data1 , ignore_index = True)
testData = testData.append(data2 , ignore_index = True)
testData = testData.append(data3 , ignore_index = True)
testData = testData.append(data4 , ignore_index = True)
testData = testData.append(data6 , ignore_index = True)
testData = testData.append(data5 , ignore_index = True)


testData['labels'] = "Disimilar"
testData['labels'][1200:1500] = "Similar"

testData

,sentence,labels
0,Art n 1 OED Online December 2011 Oxford Univer...,Disimilar
1,There is no generally agreed definition of wha...,Disimilar
2,The nature of art and related concepts such as...,Disimilar
3,In the of the history of art artistic works h...,Disimilar
4,Over time like Plato Aristotle Socrates and K...,Disimilar
...,...,...
1795,The present systematic review has several For...,Disimilar
1796,The strengths of the present include the a pr...,Disimilar
1797,Among the food groups selecting specific opti...,Disimilar
1798,Tuomilehto J Lindstrom J Eriksson JG et al Pre...,Disimilar


In [54]:
# Provide a default for the predict column
testData['predict'] = '.'

### Label the predict column based on the similarity score

In [55]:
for idx in range(len(testData)):
    line = testData['sentence'][idx]
    sent = tokenization(line)
    removal= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']

    sent = [w for w in sent if not w.lower() in stopwords.words('english')]

    # sw contains the list of stopwords
    sw = stopwords.words('english') 
    l1 =[];l2 =[]

    # remove stop words from the string
    X_set = {w for w in sent if not w in sw} 
    Y_set = {w for w in tops if not w in sw}

    # form a set containing keywords of both strings 
    rvector = X_set.union(Y_set) 
    for w in rvector:
        if w in X_set: l1.append(1) 
        else: l1.append(0)
        if w in Y_set: l2.append(1)
        else: l2.append(0)
    c = 0
    
    threshold = 0.037
    
    if cosine(rvector , c) >= threshold:
        testData['predict'][idx] = 'Similar'
        print(idx , testData['predict'][idx])
    else:
        testData['predict'][idx] = 'Disimilar'
        print(idx , testData['predict'][idx])

0 Disimilar
1 Disimilar
2 Disimilar
3 Disimilar
4 Disimilar
5 Disimilar
6 Disimilar
7 Disimilar
8 Disimilar
9 Disimilar
10 Disimilar
11 Disimilar
12 Disimilar
13 Disimilar
14 Disimilar
15 Disimilar
16 Disimilar
17 Disimilar
18 Disimilar
19 Disimilar
20 Disimilar
21 Disimilar
22 Similar
23 Similar
24 Disimilar
25 Disimilar
26 Disimilar
27 Disimilar
28 Disimilar
29 Disimilar
30 Disimilar
31 Disimilar
32 Disimilar
33 Disimilar
34 Disimilar
35 Similar
36 Disimilar
37 Disimilar
38 Disimilar
39 Similar
40 Disimilar
41 Disimilar
42 Disimilar
43 Disimilar
44 Disimilar
45 Disimilar
46 Disimilar
47 Disimilar
48 Disimilar
49 Disimilar
50 Disimilar
51 Disimilar
52 Similar
53 Disimilar
54 Disimilar
55 Disimilar
56 Disimilar
57 Similar
58 Disimilar
59 Similar
60 Similar
61 Disimilar
62 Disimilar
63 Disimilar
64 Disimilar
65 Disimilar
66 Disimilar
67 Disimilar
68 Disimilar
69 Disimilar
70 Disimilar
71 Disimilar
72 Disimilar
73 Disimilar
74 Disimilar
75 Disimilar
76 Similar
77 Disimilar
78 Disimilar
7

630 Disimilar
631 Disimilar
632 Disimilar
633 Disimilar
634 Disimilar
635 Disimilar
636 Similar
637 Similar
638 Similar
639 Disimilar
640 Disimilar
641 Disimilar
642 Disimilar
643 Disimilar
644 Disimilar
645 Disimilar
646 Disimilar
647 Disimilar
648 Disimilar
649 Disimilar
650 Disimilar
651 Disimilar
652 Disimilar
653 Disimilar
654 Disimilar
655 Disimilar
656 Disimilar
657 Disimilar
658 Disimilar
659 Disimilar
660 Disimilar
661 Similar
662 Disimilar
663 Disimilar
664 Disimilar
665 Disimilar
666 Disimilar
667 Disimilar
668 Disimilar
669 Disimilar
670 Disimilar
671 Disimilar
672 Disimilar
673 Disimilar
674 Disimilar
675 Disimilar
676 Disimilar
677 Disimilar
678 Disimilar
679 Disimilar
680 Disimilar
681 Disimilar
682 Similar
683 Disimilar
684 Disimilar
685 Disimilar
686 Disimilar
687 Similar
688 Disimilar
689 Disimilar
690 Disimilar
691 Disimilar
692 Disimilar
693 Disimilar
694 Disimilar
695 Disimilar
696 Disimilar
697 Disimilar
698 Disimilar
699 Disimilar
700 Disimilar
701 Disimilar
702 

1219 Similar
1220 Similar
1221 Disimilar
1222 Similar
1223 Disimilar
1224 Disimilar
1225 Similar
1226 Similar
1227 Disimilar
1228 Similar
1229 Disimilar
1230 Similar
1231 Similar
1232 Similar
1233 Similar
1234 Similar
1235 Similar
1236 Similar
1237 Similar
1238 Similar
1239 Similar
1240 Similar
1241 Similar
1242 Disimilar
1243 Similar
1244 Disimilar
1245 Similar
1246 Similar
1247 Similar
1248 Similar
1249 Similar
1250 Disimilar
1251 Similar
1252 Similar
1253 Similar
1254 Disimilar
1255 Similar
1256 Similar
1257 Similar
1258 Similar
1259 Similar
1260 Similar
1261 Disimilar
1262 Similar
1263 Similar
1264 Similar
1265 Similar
1266 Similar
1267 Disimilar
1268 Similar
1269 Disimilar
1270 Disimilar
1271 Disimilar
1272 Similar
1273 Disimilar
1274 Similar
1275 Similar
1276 Similar
1277 Disimilar
1278 Disimilar
1279 Similar
1280 Similar
1281 Disimilar
1282 Disimilar
1283 Disimilar
1284 Disimilar
1285 Similar
1286 Disimilar
1287 Disimilar
1288 Similar
1289 Disimilar
1290 Similar
1291 Disimilar
1

1792 Disimilar
1793 Disimilar
1794 Disimilar
1795 Disimilar
1796 Similar
1797 Disimilar
1798 Disimilar
1799 Disimilar


### Count the number of accurate predictions

In [56]:
def accuracy_score():
    accuracy = 0
    for idx in range(len(testData)):
        if testData['labels'][idx] == testData['predict'][idx]:
            accuracy += 1
    accuracy
    return accuracy / len(testData)

In [57]:
accuracy_score()

0.88